# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [29]:
from xml.etree import ElementTree as ET
import pandas as pd

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [30]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [31]:
# print names of all countries
for child in document_tree.getroot():
    print child.find('name').text

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [32]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print '* ' + element.find('name').text + ':',
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print capitals_string[:-2]

* Albania: Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece: Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia: Skopje, Kumanovo
* Serbia: Beograd, Novi Sad, Niš
* Montenegro: Podgorica
* Kosovo: Prishtine
* Andorra: Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

# 1. 10 countries with the lowest infant mortality rates

In [33]:
#Create dictionary with countries and infant mortality rates
document = ET.parse( './data/mondial_database.xml' )
root = document.getroot()
infant_mortality = {}
for country in root.iterfind('country'):
    name = country.find('name').text
    for im in country.iterfind('infant_mortality'):
        if im is not None:
            mor = im.text
            infant_mortality[name] = float(mor)

#Convert dictionary to pandas dataframe 
x = pd.DataFrame([[key,value] for key,value in infant_mortality.iteritems()],columns=["country","mortality"])
x.sort_values('mortality').head(10)

,country,mortality
35,Monaco,1.81
210,Japan,2.13
73,Norway,2.48
66,Bermuda,2.48
78,Singapore,2.53
108,Sweden,2.60
57,Czech Republic,2.63
145,Hong Kong,2.73
54,Macao,3.13
188,Iceland,3.15


# 2. 10 cities with the largest population

In [34]:
#create dictionary with country and city populations

city_pop = {}
for country in root.iterfind('country'):
    for city in country.iterfind('city'):
        if city is not None:
            name = city.find('name').text
            if city.find('population') is not None:
                pop = city.find('population').text
                city_pop[name] = float(pop)
#create pandas dataframe

pops = pd.DataFrame([[key,value] for key,value in city_pop.iteritems()],columns=["city","pop"])
pops.sort_values('pop',ascending=False).head(10)

,city,pop
318,Seoul,10229262
45,Hong Kong,7055071
376,Al Qahirah,6053000
94,Bangkok,5876000
8,Ho Chi Minh,3924435
335,Busan,3813814
75,New Taipei,3722082
230,Hanoi,3056146
287,Al Iskandariyah,2917000
102,Taipei,2626138


# 3. 10 ethnic groups with the largest overall population

In [35]:
#Create a list of lists for each ethnic group in each country
ethnic_groups = []
for country in root.iterfind('country'):
    name = country.find('name').text
    pop = country.find('population').text
    for ethnicity in country.findall('ethnicgroup'):
        x = [name, float(pop), ethnicity.text, float(ethnicity.get('percentage'))]
        ethnic_groups.append(x)
ethnic_groups 

#Convert list of lists to dataframe
eg = pd.DataFrame(ethnic_groups)
cols = ['Country','Population','Ethnicity','Percentage']
eg.columns = cols

#Find population of ethnic groups
eg['Fraction'] = eg.Percentage/100
eg['EthnicityPopulation'] = eg.Fraction*eg.Population

#Find groups with largest population using groupby
x = eg.groupby(['Ethnicity'])
y = x.EthnicityPopulation.sum()
y.sort_values(ascending=False).head(10)

Ethnicity
Han Chinese    4.975551e+08
European       1.928658e+08
Indo-Aryan     1.716454e+08
Russian        9.275844e+07
African        8.632937e+07
Japanese       8.170627e+07
German         6.623219e+07
Dravidian      5.959908e+07
English        4.231499e+07
Mestizo        3.554233e+07
Name: EthnicityPopulation, dtype: float64

# 4. Name and Country of a longest river


In [72]:

#Get a list of lists of rivers with their name, country code, and length
rivers = [] 
for river in root.iterfind('river'):
    name = river.find('name').text
    if river.get('country') is not None:
        country = river.get('country')
    if river.find('length') is not None:
        length = float(river.find('length').text)
    river = [name, country, length]
    rivers.append(river)

#Convert list of lists to dataframe and then sort the data frame based on length of river
rivers_df = pd.DataFrame(rivers, columns=['river','country','length'])
longest_rivers = rivers_df.sort('length',ascending=False)

#Find the longest river and the country code of that river
river_code = longest_rivers.iloc[0][1]
if len(river_code) > 3:
    river_code = river_code[0:2]

#Match the country code of the longest river to the actual country
for country in root.iterfind('country'):
    code = country.get('car_code')
    if code == river_code:
        code_country = country.find('name').text

print 'The longest river is the ' + str(longest_rivers.iloc[0][0]) + ' in ' + str(code_country) + ' and it is ' + str(longest_rivers.iloc[0][2]) + ' km'

    
    
    
        
       
    

The longest river is the Amazonas in Colombia and it is 6448.0 km


C:\Users\firstname.lastname\Anaconda2\lib\site-packages\ipykernel\__main__.py:15: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


## Name and Country of the largest lake by depth

In [92]:
lakes = [] 
for lake in root.iterfind('lake'):
    name = lake.find('name').text
    if lake.get('country') is not None:
        country = lake.get('country')
    if lake.find('depth') is not None:
        depth = float(lake.find('depth').text)
    lake = [name, country, depth]
    lakes.append(lake)

lakes_df = pd.DataFrame(lakes, columns=['lake','country','depth'])
largest_lakes = lakes_df.sort_values('depth',ascending=False)

lake_code = largest_lakes.iloc[0][1]
if len(lake_code) > 3:
    lake_code = lake_code[0:2]
else:
    lake_code = largest_lakes.iloc[0][1]

#Match the country code of the longest river to the actual country
for country in root.iterfind('country'):
    code = country.get('car_code')
    if code == lake_code:
        code_country = country.find('name').text

print 'The largest lake is the ' + str(largest_lakes.iloc[0][0]) + ' in ' + str(code_country) + ' and it is ' + str(largest_lakes.iloc[0][2]) + ' m'

    
    
    
        
       



The largest lake is the Ozero Baikal in Russia and it is 1637.0 m


## Name and country of airport at the highest elevation

In [91]:
airports = [] 
for airport in root.iterfind('airport'):
    name = airport.find('name').text
    if airport.get('country') is not None:
        country = airport.get('country')
    if airport.find('elevation').text is not None:
        elevation = int(airport.find('elevation').text)
        airport = [name, country, elevation]
        airports.append(airport)

air_df = pd.DataFrame(airports, columns=['airport','country','elevation'])
highest_air = air_df.sort_values('elevation',ascending=False)

air_code = highest_air.iloc[0][1]
if len(air_code) > 3:
    air_code = air_code[0:3]
else:
    air_code = highest_air.iloc[0][1]

#Match the country code of the longest river to the actual country
for country in root.iterfind('country'):
    code = country.get('car_code')
    if code == air_code:
        code_country = country.find('name').text

print 'The highest airport is ' + str(highest_air.iloc[0][0]) + ' in ' + str(code_country) + ' and it is ' + str(highest_air.iloc[0][2]) + ' m'

    
    
    
        

    

The highest airport is El Alto Intl in Bolivia and it is 4063 m


airport      El Alto Intl
country               BOL
elevation            4063
Name: 80, dtype: object